In [10]:
import argparse

import torch
import mmcv
from mmcv.runner import load_checkpoint, parallel_test, obj_from_dict
from mmcv.parallel import scatter, collate, MMDataParallel

from mmdet import datasets
from mmdet.core import results2json, coco_eval
from mmdet.datasets import build_dataloader, CocoDataset, CustomDataset
from mmdet.models import build_detector, detectors
import numpy as np
from PIL import Image

def single_test(model, data_loader, gt, show=False):
    model.eval()
    results = []
    file_names = []
    dataset = data_loader.dataset
    prog_bar = mmcv.ProgressBar(len(dataset))
    for i, data in enumerate(data_loader):
        print (i)
        with torch.no_grad():
            result = model(return_loss=False, rescale=not show, **data)
        results.append(result)
        file_names.append(gt['images'][i]["file_name"])
        if show:
            model.module.show_result(data, result, dataset.img_norm_cfg,
                                     dataset=dataset.CLASSES)
        
        batch_size = data['img'][0].size(0)
        for _ in range(batch_size):
            prog_bar.update()

    return results,file_names


In [11]:
out_file = '/home/hywel/Documents/mmdetection/result.pkl'
image_dir = '/home/hywel/Documents/mmdetection/data/coco/test_b/'
config_file = '/home/hywel/Documents/mmdetection/configs/dcn/cascade_rcnn_dconv_c3-c5_r101_fpn_1x.py'
checkpoint_file = '/home/hywel/Documents/mmdetection/work_dirs/cascade_rcnn_dconv_c3-c5_r101_fpn_1x/latest.pth'
import json
import random
from skimage.io import imsave

datasets_path = '/home/hywel/Documents/mmdetection/data/coco/annotations/'
f = open('{}test_b.json'.format(datasets_path) ,encoding='utf-8')
gt = json.load(f)

cfg = mmcv.Config.fromfile(config_file)
# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
cfg.data.test.test_mode = True

dataset = obj_from_dict(cfg.data.test, datasets, dict(test_mode=True))

model = build_detector(
    cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
load_checkpoint(model, checkpoint_file)
model = MMDataParallel(model, device_ids=[0])

data_loader = build_dataloader(
    dataset,
    imgs_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    num_gpus=1,
    dist=False,
    shuffle=False)
outputs,file_names = single_test(model, data_loader, gt,False)


    

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[                              ] 0/3398, elapsed: 0s, ETA:0
[                              ] 1/3398, 0.4 task/s, elapsed: 3s, ETA:  8842s1
[                              ] 2/3398, 0.4 task/s, elapsed: 5s, ETA:  7738s2
[                              ] 3/3398, 0.5 task/s, elapsed: 7s, ETA:  7508s3
[                              ] 4/3398, 0.4 task/s, elapsed: 9s, ETA:  7646s4
[                              ] 5/3398, 0.4 task/s, elapsed: 11s, ETA:  7603s5
[                              ] 6/3398, 0.5 task/s, elapsed: 13s, ETA:  7373s6
[                              ] 7/3398, 0.5 task/s, elapsed: 15s, ETA:  7222s7
[                              ] 8/3398, 0.5 task/s, elapsed: 17s, ETA:  7328s8
[                              ] 9/3398, 0.5 task/s, elapsed: 19s, ETA:  7214s9
[                              ] 10/3398, 0.5 task/s, elapsed: 22s, ETA:  7320s10
[                              ] 11/3398, 0.5 task/s, elaps

[                              ] 98/3398, 0.5 task/s, elapsed: 212s, ETA:  7142s98
[                              ] 99/3398, 0.5 task/s, elapsed: 215s, ETA:  7149s99
[                              ] 100/3398, 0.5 task/s, elapsed: 217s, ETA:  7144s100
[                              ] 101/3398, 0.5 task/s, elapsed: 219s, ETA:  7150s101
[                              ] 102/3398, 0.5 task/s, elapsed: 221s, ETA:  7153s102
[                              ] 103/3398, 0.5 task/s, elapsed: 224s, ETA:  7153s103
[                              ] 104/3398, 0.5 task/s, elapsed: 226s, ETA:  7144s104
[                              ] 105/3398, 0.5 task/s, elapsed: 228s, ETA:  7140s105
[                              ] 106/3398, 0.5 task/s, elapsed: 230s, ETA:  7133s106
[                              ] 107/3398, 0.5 task/s, elapsed: 232s, ETA:  7131s107
[                              ] 108/3398, 0.5 task/s, elapsed: 234s, ETA:  7136s108
[                              ] 109/3398, 0.5 task/s, elapsed: 236s,

[>                             ] 194/3398, 0.5 task/s, elapsed: 417s, ETA:  6893s194
[>                             ] 195/3398, 0.5 task/s, elapsed: 420s, ETA:  6891s195
[>                             ] 196/3398, 0.5 task/s, elapsed: 422s, ETA:  6890s196
[>                             ] 197/3398, 0.5 task/s, elapsed: 423s, ETA:  6876s197
[>                             ] 198/3398, 0.5 task/s, elapsed: 425s, ETA:  6873s198
[>                             ] 199/3398, 0.5 task/s, elapsed: 428s, ETA:  6872s199
[>                             ] 200/3398, 0.5 task/s, elapsed: 429s, ETA:  6867s200
[>                             ] 201/3398, 0.5 task/s, elapsed: 432s, ETA:  6869s201
[>                             ] 202/3398, 0.5 task/s, elapsed: 434s, ETA:  6865s202
[>                             ] 203/3398, 0.5 task/s, elapsed: 436s, ETA:  6861s203
[>                             ] 204/3398, 0.5 task/s, elapsed: 438s, ETA:  6862s204
[>                             ] 205/3398, 0.5 task/s, elapsed: 4

[>>                            ] 290/3398, 0.5 task/s, elapsed: 624s, ETA:  6686s290
[>>                            ] 291/3398, 0.5 task/s, elapsed: 626s, ETA:  6686s291
[>>                            ] 292/3398, 0.5 task/s, elapsed: 628s, ETA:  6682s292
[>>                            ] 293/3398, 0.5 task/s, elapsed: 631s, ETA:  6683s293
[>>                            ] 294/3398, 0.5 task/s, elapsed: 633s, ETA:  6678s294
[>>                            ] 295/3398, 0.5 task/s, elapsed: 635s, ETA:  6680s295
[>>                            ] 296/3398, 0.5 task/s, elapsed: 638s, ETA:  6681s296
[>>                            ] 297/3398, 0.5 task/s, elapsed: 640s, ETA:  6680s297
[>>                            ] 298/3398, 0.5 task/s, elapsed: 642s, ETA:  6678s298
[>>                            ] 299/3398, 0.5 task/s, elapsed: 644s, ETA:  6679s299
[>>                            ] 300/3398, 0.5 task/s, elapsed: 646s, ETA:  6674s300
[>>                            ] 301/3398, 0.5 task/s, elapsed: 6

[>>>                           ] 386/3398, 0.5 task/s, elapsed: 832s, ETA:  6490s386
[>>>                           ] 387/3398, 0.5 task/s, elapsed: 834s, ETA:  6490s387
[>>>                           ] 388/3398, 0.5 task/s, elapsed: 836s, ETA:  6485s388
[>>>                           ] 389/3398, 0.5 task/s, elapsed: 838s, ETA:  6484s389
[>>>                           ] 390/3398, 0.5 task/s, elapsed: 840s, ETA:  6482s390
[>>>                           ] 391/3398, 0.5 task/s, elapsed: 842s, ETA:  6479s391
[>>>                           ] 392/3398, 0.5 task/s, elapsed: 845s, ETA:  6477s392
[>>>                           ] 393/3398, 0.5 task/s, elapsed: 847s, ETA:  6473s393
[>>>                           ] 394/3398, 0.5 task/s, elapsed: 849s, ETA:  6473s394
[>>>                           ] 395/3398, 0.5 task/s, elapsed: 851s, ETA:  6470s395
[>>>                           ] 396/3398, 0.5 task/s, elapsed: 854s, ETA:  6471s396
[>>>                           ] 397/3398, 0.5 task/s, elapsed: 8

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3004/3398, 0.5 task/s, elapsed: 6472s, ETA:   849s3004
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3005/3398, 0.5 task/s, elapsed: 6474s, ETA:   847s3005
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3006/3398, 0.5 task/s, elapsed: 6476s, ETA:   845s3006
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3007/3398, 0.5 task/s, elapsed: 6478s, ETA:   842s3007
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3008/3398, 0.5 task/s, elapsed: 6481s, ETA:   840s3008
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3009/3398, 0.5 task/s, elapsed: 6483s, ETA:   838s3009
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3010/3398, 0.5 task/s, elapsed: 6485s, ETA:   836s3010
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3011/3398, 0.5 task/s, elapsed: 6487s, ETA:   834s3011
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3012/3398, 0.5 task/s, elapsed: 6489s, ETA:   832s3012
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3013/3398, 0.5 task/s, elapsed: 6491s, ETA:   829s3013
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ] 3014/3398, 0.5 task/s, elapsed: 6493s, ETA:   827s3014
[>>>>>>>>>>>>>>>>>>>>>>>>>>    ]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3097/3398, 0.5 task/s, elapsed: 6673s, ETA:   649s3097
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3098/3398, 0.5 task/s, elapsed: 6676s, ETA:   646s3098
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3099/3398, 0.5 task/s, elapsed: 6678s, ETA:   644s3099
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3100/3398, 0.5 task/s, elapsed: 6680s, ETA:   642s3100
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3101/3398, 0.5 task/s, elapsed: 6682s, ETA:   640s3101
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3102/3398, 0.5 task/s, elapsed: 6684s, ETA:   638s3102
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3103/3398, 0.5 task/s, elapsed: 6686s, ETA:   636s3103
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3104/3398, 0.5 task/s, elapsed: 6688s, ETA:   633s3104
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3105/3398, 0.5 task/s, elapsed: 6690s, ETA:   631s3105
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3106/3398, 0.5 task/s, elapsed: 6692s, ETA:   629s3106
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ] 3107/3398, 0.5 task/s, elapsed: 6695s, ETA:   627s3107
[>>>>>>>>>>>>>>>>>>>>>>>>>>>   ]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3190/3398, 0.5 task/s, elapsed: 6871s, ETA:   448s3190
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3191/3398, 0.5 task/s, elapsed: 6873s, ETA:   446s3191
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3192/3398, 0.5 task/s, elapsed: 6875s, ETA:   444s3192
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3193/3398, 0.5 task/s, elapsed: 6878s, ETA:   442s3193
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3194/3398, 0.5 task/s, elapsed: 6880s, ETA:   439s3194
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3195/3398, 0.5 task/s, elapsed: 6882s, ETA:   437s3195
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3196/3398, 0.5 task/s, elapsed: 6884s, ETA:   435s3196
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3197/3398, 0.5 task/s, elapsed: 6886s, ETA:   433s3197
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3198/3398, 0.5 task/s, elapsed: 6889s, ETA:   431s3198
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3199/3398, 0.5 task/s, elapsed: 6891s, ETA:   429s3199
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3200/3398, 0.5 task/s, elapsed: 6893s, ETA:   426s3200
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3283/3398, 0.5 task/s, elapsed: 7067s, ETA:   248s3283
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>  ] 3284/3398, 0.5 task/s, elapsed: 7069s, ETA:   245s3284
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3285/3398, 0.5 task/s, elapsed: 7072s, ETA:   243s3285
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3286/3398, 0.5 task/s, elapsed: 7074s, ETA:   241s3286
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3287/3398, 0.5 task/s, elapsed: 7076s, ETA:   239s3287
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3288/3398, 0.5 task/s, elapsed: 7078s, ETA:   237s3288
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3289/3398, 0.5 task/s, elapsed: 7081s, ETA:   235s3289
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3290/3398, 0.5 task/s, elapsed: 7082s, ETA:   232s3290
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3291/3398, 0.5 task/s, elapsed: 7085s, ETA:   230s3291
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3292/3398, 0.5 task/s, elapsed: 7087s, ETA:   228s3292
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3293/3398, 0.5 task/s, elapsed: 7089s, ETA:   226s3293
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3376/3398, 0.5 task/s, elapsed: 7269s, ETA:    47s3376
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3377/3398, 0.5 task/s, elapsed: 7271s, ETA:    45s3377
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3378/3398, 0.5 task/s, elapsed: 7273s, ETA:    43s3378
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3379/3398, 0.5 task/s, elapsed: 7274s, ETA:    41s3379
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3380/3398, 0.5 task/s, elapsed: 7276s, ETA:    39s3380
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3381/3398, 0.5 task/s, elapsed: 7278s, ETA:    37s3381
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3382/3398, 0.5 task/s, elapsed: 7281s, ETA:    34s3382
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3383/3398, 0.5 task/s, elapsed: 7282s, ETA:    32s3383
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3384/3398, 0.5 task/s, elapsed: 7285s, ETA:    30s3384
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3385/3398, 0.5 task/s, elapsed: 7287s, ETA:    28s3385
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 3386/3398, 0.5 task/s, elapsed: 7289s, ETA:    26s3386
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ]

In [13]:
sub_dict = {}
sub_dict['results'] = []
for result,img_name in zip(outputs,file_names ):
    img_path = image_dir+img_name
    image = Image.open(img_path)
    (im_width, im_height) = image.size
    image_np=load_image_into_numpy_array(image)
    boxes,labels,scores = mmresult_to_boxes_labels(result)
    #overlay_boxes(image_np, boxes,labels)
    
    #print (boxes)
    image_dict = {}
    image_dict['filename'] = img_name
    image_dict['rects'] = []
    for box, score, label in zip(boxes, scores, labels):
        # scores are sorted so we can break
        print (box, score, label)
        if score < 0.5:
            break
        box_dict = {}
        color = 'red' #label_color(label)
        box_dict['xmin'] = int(box[0])
        box_dict['ymin'] = int(box[1])
        box_dict['xmax'] = int(box[2])
        box_dict['ymax'] = int(box[3])
        box_dict['label'] = int(label)
        box_dict["confidence"] = float(score)
        image_dict['rects'].append(box_dict)
            
    sub_dict['results'].append(image_dict)
    
with open("record_b.json","w") as f:
    json.dump(sub_dict,f)
    print("加载入文件完成...")
    

[206.21417, 238.17828, 322.6141, 327.2368] 0.8465041 3
[183.23991, 259.0049, 275.95837, 360.3844] 0.79381335 3
[601.7155, 277.9029, 651.6813, 293.50537] 0.92139536 4
[608.909, 283.36383, 656.96625, 297.48096] 0.83198386 4
[613.9191, 251.63425, 638.92224, 277.74237] 0.69559705 4
[602.61633, 252.6838, 630.9618, 280.61658] 0.6930629 4
[630.3475, 252.18698, 656.4015, 277.42328] 0.65331423 4
[238.86415, 185.84291, 339.88422, 251.36888] 0.72254264 5
[271.76996, 158.86197, 376.91028, 230.06982] 0.6520548 5
[313.36395, 592.4385, 424.18994, 618.1078] 0.64370376 4
[718.82275, 91.202866, 762.689, 130.88144] 0.86643875 4
[54.34011, 199.42743, 107.77048, 287.88416] 0.7811285 4
[164.92136, 17.193245, 236.12262, 57.3175] 0.73442173 4
[241.38992, 281.65497, 300.27698, 313.95996] 0.7078544 4
[699.50977, 88.360275, 761.04785, 119.31756] 0.7020023 4
[356.82278, 45.803555, 419.29718, 99.217674] 0.68837416 4
[867.7744, 195.77045, 963.1393, 254.79825] 0.6293807 1
[766.8729, 379.71423, 852.6853, 452.09775] 0

[209.41833, 174.70853, 274.9402, 222.42807] 0.8736424 2
[116.71236, 198.50119, 194.33658, 241.47456] 0.8480922 2
[157.77621, 204.8977, 220.39508, 250.82758] 0.83350897 2
[192.01443, 182.34647, 245.7942, 245.92068] 0.7913057 2
[137.62537, 170.79193, 217.58151, 203.76265] 0.7505989 2
[217.01112, 161.08571, 274.3324, 200.62143] 0.6762266 2
[165.71512, 64.502945, 301.1902, 139.47977] 0.98871136 3
[397.47546, 177.5458, 493.22766, 310.0] 0.94183445 3
[660.4833, 224.7806, 866.06946, 322.04788] 0.9255848 5
[245.96706, 194.07962, 277.38104, 215.08182] 0.7294885 4
[359.9751, 158.91019, 384.7154, 256.7536] 0.8732292 3
[75.925735, 272.0302, 253.62186, 375.72632] 0.94996524 1
[321.3073, 288.37903, 499.99554, 395.68814] 0.9379574 1
[361.0333, 148.78062, 531.97485, 251.06447] 0.88063633 1
[111.32003, 132.99393, 293.5275, 232.07025] 0.8312748 1
[300.11893, 307.21606, 357.15656, 353.4978] 0.9447861 2
[837.0588, 382.67398, 922.80133, 414.52588] 0.92303133 2
[837.9595, 383.56262, 911.27716, 401.09204] 0.

[294.06754, 305.90875, 409.02228, 401.34558] 0.95880204 1
[268.36264, 356.62048, 337.4709, 414.0564] 0.86711407 1
[283.88824, 351.09027, 426.12585, 407.27805] 0.7089026 1
[217.13565, 237.6325, 265.3996, 272.06647] 0.98256767 4
[124.59504, 370.03278, 149.2641, 394.6551] 0.94130695 4
[164.09915, 386.143, 178.6803, 414.89105] 0.8576479 4
[149.16275, 369.30765, 175.57922, 399.609] 0.8455788 4
[152.186, 366.9892, 164.78363, 395.11572] 0.7181779 4
[131.76614, 398.75906, 164.76418, 423.8006] 0.70700777 4
[109.63395, 382.66592, 148.98257, 400.8051] 0.62761605 4
[931.3292, 313.1576, 1031.6432, 348.56192] 0.9460242 2
[691.05505, 268.51794, 877.59174, 305.28284] 0.82115054 5
[274.27008, 368.93372, 390.82388, 400.76584] 0.703995 5
[121.381226, 144.74883, 253.8742, 209.65451] 0.7462457 3
[318.63205, 86.16541, 339.5699, 106.559555] 0.66394746 4
[1011.3893, 395.04337, 1080.1324, 428.79654] 0.9960569 1
[857.7031, 298.66998, 930.1105, 374.43152] 0.9674532 3
[739.2243, 446.19473, 831.0217, 537.34985] 0.

[532.98456, 193.24214, 803.55493, 362.99536] 0.8451324 4
[463.52313, 103.616325, 508.8851, 170.11172] 0.7685149 3
[530.6314, 106.442154, 604.4405, 301.7212] 0.9237384 4
[77.550896, 250.41943, 172.77621, 416.16125] 0.71151054 4
[428.92554, 268.50372, 538.6292, 440.91226] 0.6788356 4
[323.63217, 278.5236, 419.31635, 432.9194] 0.6713286 4
[186.64407, 263.93335, 281.71667, 425.90826] 0.65190077 4
[601.7627, 43.756622, 791.8351, 377.3604] 0.64444685 4
[342.33862, 151.9751, 387.12994, 201.832] 0.89490014 2
[732.62885, 177.74255, 820.0165, 206.61487] 0.8611708 2
[742.4735, 190.11072, 821.5189, 231.69516] 0.81069165 2
[324.04016, 142.84677, 383.44293, 210.65778] 0.6350577 2
[506.62057, 130.14531, 613.0015, 216.80074] 0.9308977 5
[839.68896, 298.62714, 981.6279, 382.4206] 0.9162859 5
[458.0981, 164.9973, 575.48816, 255.77652] 0.7121011 5
[199.25012, 241.46146, 236.32452, 281.67834] 0.6209018 4
[379.88727, 121.43652, 722.52356, 277.93784] 0.7928661 4
[430.7063, 423.49347, 604.56946, 517.368] 0.6

[24.263004, 210.14253, 98.34332, 241.69965] 0.99159765 1
[649.91406, 407.9944, 736.6308, 426.25653] 0.81351817 2
[653.0659, 392.4167, 741.9195, 411.3781] 0.7936917 2
[651.19507, 378.52734, 734.9041, 424.2795] 0.7158621 2
[111.32132, 367.97012, 177.56473, 440.48065] 0.9589989 4
[331.97983, 317.83148, 366.72812, 341.6264] 0.8478441 4
[369.46722, 323.04535, 417.64407, 365.1872] 0.84339464 4
[335.0748, 339.6048, 387.7392, 359.97485] 0.79455614 4
[376.36154, 319.94763, 423.80655, 333.14008] 0.77090454 4
[385.65652, 333.38867, 418.57706, 363.80573] 0.6545218 4
[359.62842, 362.0204, 413.04456, 373.2574] 0.61248076 4
[268.5113, 91.29224, 319.53827, 132.1186] 0.62414634 4
[234.32469, 83.43694, 326.2647, 161.94757] 0.6008995 4
[126.2216, 62.27904, 245.93289, 113.52026] 0.7351996 4
[107.18512, 17.924618, 170.10751, 43.674137] 0.6516981 1
[175.30807, 328.97464, 201.28662, 348.38232] 0.8605664 4
[235.13504, 127.72356, 298.79852, 182.0369] 0.96224046 1
[74.97923, 224.17657, 115.756645, 301.0581] 0.8

...

[3.0649314, 63.137924, 125.31478, 233.89853] 0.7093481 3
[534.36633, 361.10168, 564.414, 388.0611] 0.7795887 4
[385.83102, 289.36154, 462.19788, 421.84747] 0.605732 5
[46.863586, 103.1664, 126.430954, 180.53873] 0.6009027 4
[191.67722, 46.33446, 225.65349, 70.50746] 0.604368 4
[446.1771, 412.71234, 483.26703, 457.06384] 0.77762675 4
[726.2931, 401.6734, 762.8336, 439.4336] 0.69173676 4
[293.3936, 51.132828, 327.86954, 88.81818] 0.68476003 4
[351.1353, 63.51762, 373.76062, 84.29431] 0.66143537 4
[192.98524, 343.1175, 350.7732, 394.7091] 0.7071688 3
[196.46527, 219.83374, 296.66248, 268.66046] 0.6455424 3
[378.0747, 185.8385, 416.982, 236.42505] 0.68278325 2
[184.18646, 179.29216, 296.98285, 262.38013] 0.852466 3
[349.41385, 174.84227, 400.79578, 216.2486] 0.99384594 1
[222.29457, 199.55225, 274.1101, 238.745] 0.7375132 4
[274.20343, 351.53848, 301.5537, 375.41928] 0.68734837 4
[533.48425, 169.14746, 625.76636, 214.27408] 0.9742676 3
[851.8984, 390.98352, 926.84625, 413.66736] 0.8990649 

[0.0, 57.34742, 49.124443, 227.93694] 0.8787864 4
[219.25679, 51.58309, 316.2512, 218.70493] 0.7341792 4
[354.70264, 42.028755, 456.08688, 213.58823] 0.7275685 4
[86.62094, 51.78035, 186.00742, 228.04724] 0.6118623 4
[326.4627, 221.1036, 418.86908, 282.76962] 0.6909104 4
[62.570847, 67.98066, 126.61467, 172.92998] 0.6388805 3
[198.93388, 202.93771, 292.22815, 243.21912] 0.9264176 2
[232.84372, 208.23947, 287.70297, 241.93045] 0.7950343 2
[128.43938, 212.99542, 164.18442, 236.27744] 0.95814323 4
[128.74081, 202.1153, 165.90979, 226.45926] 0.9240028 4
[166.80153, 197.0715, 207.56311, 220.43822] 0.6270886 4
[0.0, 250.13123, 59.13251, 358.09033] 0.6821766 4
[235.69958, 158.69443, 287.2257, 205.96191] 0.964427 2
[264.5158, 188.45834, 359.7708, 224.27339] 0.954376 2
[275.79626, 138.32921, 314.54224, 191.08638] 0.8574046 2
[217.33817, 165.12778, 273.9552, 211.69574] 0.7911619 2
[250.3531, 144.1195, 329.0648, 184.9228] 0.7225425 2
[1004.63464, 241.12584, 1098.2528, 291.23148] 0.981044 3
[53.67

[491.43225, 220.79333, 533.3178, 313.95972] 0.9784826 3
[222.12578, 70.29573, 254.94, 96.42425] 0.984985 4
[227.7995, 65.631714, 270.85272, 95.07206] 0.8504901 4
[258.06055, 71.60414, 273.0265, 104.05387] 0.7362338 4
[262.94397, 184.02837, 284.41992, 209.42043] 0.6731878 4
[86.43723, 397.29575, 154.36276, 454.13507] 0.77105516 3
[172.54747, 170.55774, 188.59262, 186.71355] 0.7773249 4
[178.47273, 152.02823, 194.84769, 167.55579] 0.7202774 4
[144.31688, 59.47374, 159.90582, 72.829025] 0.718999 4
[116.33297, 42.915108, 145.29724, 52.57273] 0.6903738 4
[98.056915, 146.29504, 113.93379, 163.3787] 0.62930393 4
[257.33276, 64.42218, 397.47073, 143.71156] 0.9845508 3
[136.56079, 192.32411, 224.75806, 265.48987] 0.66066957 3
[165.43471, 218.11658, 284.00485, 297.30725] 0.6505308 4
[213.25668, 59.49968, 229.75443, 74.935814] 0.7431717 4
[81.65513, 26.751625, 135.02141, 68.00911] 0.6275129 3
[255.83788, 104.65582, 275.35938, 127.03856] 0.9296502 4
[355.60516, 63.448326, 372.54156, 88.68387] 0.82

[121.52373, 194.45432, 150.03412, 209.79234] 0.7583314 4
[442.54672, 237.71112, 470.63806, 274.71988] 0.63590145 4
[136.64789, 28.801235, 173.68979, 47.26573] 0.887956 4
[231.63489, 31.074867, 271.3689, 51.0293] 0.86375535 4
[178.3699, 128.25122, 213.9716, 154.79889] 0.79595035 4
[78.15671, 101.28148, 117.57768, 122.63025] 0.7799655 4
[76.47603, 123.557, 113.75287, 148.6995] 0.7625703 4
[803.86707, 74.63339, 867.9679, 110.86304] 0.9247552 1
[558.374, 564.1021, 623.4049, 657.26404] 0.7009154 4
[141.60645, 273.91467, 306.1048, 321.8545] 0.8582916 5
[143.17752, 85.36421, 179.69836, 103.879074] 0.82686555 4
[3.0950959, 56.965157, 40.29773, 74.82237] 0.7400893 4
[117.346664, 119.793755, 222.11444, 187.68025] 0.98215854 3
[84.19194, 226.03644, 259.57562, 321.0207] 0.8628078 5
[136.34595, 192.82953, 185.3305, 206.30492] 0.6817275 4
[143.59311, 205.7362, 180.28362, 220.21707] 0.6039511 4
[288.78253, 548.32947, 321.28763, 600.2194] 0.7711377 2
[325.9049, 543.9454, 370.50745, 594.8159] 0.6237473

[275.39758, 169.87558, 354.60052, 276.9295] 0.82129824 3
[396.99835, 258.87622, 422.53238, 283.21625] 0.72955287 4
[750.27954, 291.019, 891.8877, 451.758] 0.6980938 4
[230.76047, 64.582535, 412.101, 141.33931] 0.6412099 3
[175.09317, 283.04297, 287.59393, 335.21558] 0.7621731 4
[592.8197, 182.90085, 711.16364, 238.08167] 0.72261655 4
[172.55933, 319.7505, 287.03613, 368.18402] 0.6753984 4
[238.91934, 211.59045, 353.60938, 260.48975] 0.653039 4
[480.22867, 347.6207, 589.4989, 404.90466] 0.64256465 4
[862.0475, 269.4209, 911.92883, 315.19336] 0.91907287 2
[872.09424, 277.3548, 943.9779, 322.77032] 0.848827 2
[936.7307, 311.3092, 1083.7798, 361.86554] 0.98420423 3
[349.52704, 119.308914, 403.58887, 138.77254] 0.95552605 4
[870.85376, 152.92175, 1064.442, 218.30672] 0.9023201 5
[876.70935, 200.2859, 1060.6086, 261.7748] 0.88203394 5
[836.42017, 165.098, 1036.2817, 237.04884] 0.8081049 5
[762.45886, 179.44763, 788.07355, 242.88513] 0.8646919 2
[726.94946, 177.34122, 776.1825, 246.7115] 0.67

In [12]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
color_dict = {1:(255,0,0),2:(0,255,0),3:(0,0,255),4:(255,255,0),5:(0,255,255)}
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def overlay_boxes(image, boxes,labels):
    """
    Adds the predicted boxes on top of the image

    Arguments:
        image (np.ndarray): an image as returned by OpenCV
        predictions (BoxList): the result of the computation by the model.
            It should contain the field `labels`.
    """
#     labels = predictions.get_field("labels")
#     boxes = predictions.bbox

    colors = [color_dict[int(label)] for label in labels]

    for box, color in zip(boxes, colors):
        box = [int(i) for i in box]
        top_left, bottom_right = box[:2], box[2:]
        cv2.rectangle(
            image, tuple(top_left), tuple(bottom_right), color, 2
        )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()
    return image

def mmresult_to_boxes_labels(result):
    boxes = []
    labels = []
    scores = []
    for i in range(5):
        for one in result[i]:
            tmp = list(one)
            if tmp[4]>0.60:
                boxes.append(tmp[:4])
                scores.append(tmp[4])
                labels.append(i+1)
    return boxes,labels,scores
    


In [ ]:
file_names

In [ ]:
len(sub_dict['results'])